In [60]:
import datetime
import json
import pandas as pd
import plotly.express as px

In [61]:
# Example activities to track
acts_to_track = ["youtube.com",
                 "mail.google.com",
                 "localhost:8888", # For Jupyter Notebooks
                 "localhost:8889",
                 "Visual Studio",
                 "Excel"]

def activityHandler(activityInput,activitiesToTrack):
    activityOutput = ""
    if activityInput in activitiesToTrack:
        # Special catch clause for jupyter notebooks
        if (activityInput == "localhost:8888") or (activityInput == "localhost:8889"):
            activityOutput = "Python"
        else:
            activityOutput = activityInput
    else:
        # Check for containing key words kind of thing to keep track of a spec
        for i in range(len(activitiesToTrack)):
            if str(activitiesToTrack[i]) in activityInput:
                activityOutput = activitiesToTrack[i]
        
        if activityOutput == "":
            activityOutput = "Other"
            
    return activityOutput


# Total Worked

The following blocks of code can be used to get the amount of hours that you spent on a specific task along with the rate, so that you could find out how much should be charged for work done.

In [62]:
# Example rate
rate = 100

In [63]:
with open("activities.json") as f:
    data = json.load(f)
    
no_activities = len(data['activities'])

sec_arr = []
min_arr = []
hour_arr = []
day_arr = []
combo_arr = []
activity_arr = []

for j in range(no_activities):
    
    activity = data['activities'][j]['name']
    
    # Retrieving all data that specific things were worked on
    time_entry_data = (data['activities'][j]['time_entries'])
    days = 0
    hours = 0
    minutes = 0
    seconds = 0
    for i in range(len(time_entry_data)):
        days += (time_entry_data[i]['days'])
        hours += (time_entry_data[i]['hours'])
        minutes += (time_entry_data[i]['minutes'])
        seconds += (time_entry_data[i]['seconds'])
    # -------------------- Tallying up the time ----------------------- #
    count_mins = 0

    bflag = True
    while bflag:
        if seconds - 60 > 60:
            count_mins +=1
            seconds -= 60
        elif seconds > 60:
            count_mins +=1
            seconds -= 60
            bflag = False
        else:
            break

    minutes += count_mins

    count_hours = 0
    bflag = True
    while bflag:
        if minutes - 60 > 60:
            count_hours +=1
            minutes -= 60
        elif minutes > 60:
            count_hours +=1
            minutes -= 60
            bflag = False
        else:
            break

    hours += count_hours

    count_days = 0
    bflag = True
    while bflag:
        if hours - 24 > 24:
            count_days +=1
            hours -= 24
        elif hours > 24:
            count_days +=1
            hours -= 24
            bflag = False
        else:
            break

    days += count_days
    # ----------------- End Tallying up the time -------------------- #
    
    # Changing the name of specific activities
    activity_arr.append(activityHandler(activityInput = activity,
                                        activitiesToTrack = acts_to_track))
     
    # ----- Prepping for the data frame
    sec_arr.append(seconds)
    min_arr.append(minutes)
    hour_arr.append(hours)
    day_arr.append(days)
    
    # --- Combo to get everthing to the same units
    combo_arr.append(days*24 + hours + minutes/60 + seconds/3600 )
    
dataf = {
    'Activity':activity_arr,
    'Days':day_arr,
    'Hours':hour_arr,
    'Minutes':min_arr,
    'Seconds':sec_arr,
    'Total Hours': combo_arr
}

df_total = pd.DataFrame(dataf)
total_Hours_Worked = sum(combo_arr)

total_earnings = df_total['Total Hours'].sum() * rate # Earnings for the entire time spent working

df_total_new = df_total.groupby(['Activity']).sum().reset_index()

fig = px.bar(df_total_new, x='Activity', y='Total Hours')
# fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, 
                  uniformtext_mode='hide',
                  title = 'Total Application Usage Time')
fig.show()

fig = px.pie(df_total_new, values='Total Hours', names='Activity',title=f'Time Spent on different tasks: The total no. of hours worked is {total_Hours_Worked}')
fig.show()

# Keeping track of the amount worked over a period

In [64]:
# --- Check the month that specific activities occurred in
with open("activities.json") as f:
    data = json.load(f)
    
# Can Track period data
def periodTimeHandler(periodArr):
    days = 0
    hours = 0
    minutes = 0
    seconds = 0
    for i in range(len(periodArr)):
        for j in range(len(periodArr)):
            if   j == 0:
                seconds += periodArr[i][j]
            elif j == 1:
                minutes += periodArr[i][j]
            elif j == 2:
                hours   += periodArr[i][j]
            elif j == 3:
                days    += periodArr[i][j]

    # -------------------- Tallying up the time ----------------------- # 
    # need to adapt this to the array format
    count_mins = 0

    bflag = True
    while bflag:
        if seconds - 60 > 60:
            count_mins +=1
            seconds -= 60
        elif seconds > 60:
            count_mins +=1
            seconds -= 60
            bflag = False
        else:
            break

    minutes += count_mins

    count_hours = 0
    bflag = True
    while bflag:
        if minutes - 60 > 60:
            count_hours +=1
            minutes -= 60
        elif minutes > 60:
            count_hours +=1
            minutes -= 60
            bflag = False
        else:
            break

    hours += count_hours

    count_days = 0
    bflag = True
    while bflag:
        if hours - 24 > 24:
            count_days +=1
            hours -= 24
        elif hours > 24:
            count_days +=1
            hours -= 24
            bflag = False
        else:
            break

    days += count_days
    # ----------------- End Tallying up the time -------------------- #
    return (days*24 + hours + minutes/60 + seconds/3600 )

def timeHandler(time_entry_data,i):
    days = 0
    hours = 0
    minutes = 0
    seconds = 0
        
    days += (time_entry_data[i]['days'])
    hours += (time_entry_data[i]['hours'])
    minutes += (time_entry_data[i]['minutes'])
    seconds += (time_entry_data[i]['seconds'])
    
    return (seconds,minutes,hours,days)

# ============================================================================ #
# Receive input to the date you want to be able to track data from 
start_date = datetime.datetime(2020,11,10)
end_date = datetime.datetime(2020,11,12)

no_activities = len(data['activities'])
combo_arr = []
period_arr = []
date_arr = []
time_worked = []
charged_arr = []
activity_arr = []

for j in range(no_activities):
    
    activity = data['activities'][j]['name']
    
    # Retrieving all data that specific things were worked on
    time_entry_data = (data['activities'][j]['time_entries'])
    
    for i in range(len(time_entry_data)):
        date = (time_entry_data[i]['end_time'][0:10])
        date_time_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
        if start_date <= date_time_obj <= end_date:
            # can manage activities from here
            activity_arr.append(activityHandler(activityInput = activity,
                                        activitiesToTrack = acts_to_track))
                
            period_arr.append(timeHandler(time_entry_data,i)) 
            date_arr.append(date)

for i in range(len(period_arr)):
    time_worked.append(periodTimeHandler([period_arr[i]]))
    charged_arr.append(periodTimeHandler([period_arr[i]])*rate)
    
dataf = {
    'Date': date_arr,
    'Hours': time_worked,
    'Charge': charged_arr,
    'Activity':activity_arr,
}

df_total = pd.DataFrame(dataf)
df_total_new = df_total.groupby(['Activity','Date']).sum().reset_index() # This could work

# ============= Display Data ==============#
fig = px.bar(df_total_new, x='Activity', y='Hours', text='Charge')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, 
                  uniformtext_mode='hide',
                  title = 'Hours worked over period')
fig.show()

fig1 = px.bar(df_total_new, x='Date', y='Hours', text='Charge')
fig1.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig1.update_layout(uniformtext_minsize=8, 
                  uniformtext_mode='hide',
                  title = 'Hours worked over period')
fig1.show()

# Keep Track of amount of work you have done today

In [65]:
with open("activities.json") as f:
    data = json.load(f)

no_activities = len(data['activities'])

sec_arr = []
min_arr = []
hour_arr = []
day_arr = []
combo_arr = []
activity_arr = []

for j in range(no_activities):
    
    activity = data['activities'][j]['name']
    
    # Retrieving all data that specific things were worked on
    time_entry_data = (data['activities'][j]['time_entries'])
    days = 0
    hours = 0
    minutes = 0
    seconds = 0
    for i in range(len(time_entry_data)):
        if str(datetime.datetime.now())[0:10] == str(time_entry_data[i]['start_time'][0:10]):
            days += (time_entry_data[i]['days'])
            hours += (time_entry_data[i]['hours'])
            minutes += (time_entry_data[i]['minutes'])
            seconds += (time_entry_data[i]['seconds'])
    # -------------------- Tallying up the time ----------------------- #
    count_mins = 0

    bflag = True
    while bflag:
        if seconds - 60 > 60:
            count_mins +=1
            seconds -= 60
        elif seconds > 60:
            count_mins +=1
            seconds -= 60
            bflag = False
        else:
            break

    minutes += count_mins

    count_hours = 0
    bflag = True
    while bflag:
        if minutes - 60 > 60:
            count_hours +=1
            minutes -= 60
        elif minutes > 60:
            count_hours +=1
            minutes -= 60
            bflag = False
        else:
            break

    hours += count_hours

    count_days = 0
    bflag = True
    while bflag:
        if hours - 24 > 24:
            count_days +=1
            hours -= 24
        elif hours > 24:
            count_days +=1
            hours -= 24
            bflag = False
        else:
            break

    days += count_days
    # ----------------- End Tallying up the time -------------------- #
    
    if (days > 0 or minutes>0 or hours>0 or seconds > 0):
        # Changing the name of specific activities
        activity_arr.append(activityHandler(activityInput = activity,
                                        activitiesToTrack = acts_to_track))    
        # ----- Prepping for the data frame
        sec_arr.append(seconds)
        min_arr.append(minutes)
        hour_arr.append(hours)
        day_arr.append(days)

        # --- Combo to get everthing to the same units
        combo_arr.append(days*24 + hours + minutes/60 + seconds/3600 )
        
dataf = {
    'Activity':activity_arr,
    'Days':day_arr,
    'Hours':hour_arr,
    'Minutes':min_arr,
    'Seconds':sec_arr,
    'Total Hours': combo_arr
}

df_total = pd.DataFrame(dataf)
total_Hours_Worked = sum(combo_arr)

# ================ Display Data ====================#
fig = px.bar(df_total, x='Activity', y='Total Hours')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

fig1 = px.pie(df_total, values='Total Hours', 
             names='Activity',
             title=f'Time Spent on different tasks: The total no. of hours worked is {total_Hours_Worked}')
fig1.update_layout(uniformtext_minsize=4, uniformtext_mode='hide')
fig1.show()

# Checking Monthly Data 

In [66]:
# --- Check the month that specifc activities occurred in
with open("activities.json") as f:
    data = json.load(f)

In [67]:
def timeHandler(time_entry_data,i):
    days = 0
    hours = 0
    minutes = 0
    seconds = 0
        
    days += (time_entry_data[i]['days'])
    hours += (time_entry_data[i]['hours'])
    minutes += (time_entry_data[i]['minutes'])
    seconds += (time_entry_data[i]['seconds'])
    return (seconds,minutes,hours,days)

def monthTimeHandler(month_arr):
    days = 0
    hours = 0
    minutes = 0
    seconds = 0
    for i in range(len(month_arr)):
        for j in range(len(month_arr)):
            if j == 0:
                seconds += month_arr[i][j]
            elif j==1:
                minutes += month_arr[i][j]
            elif j ==2:
                hours += month_arr[i][j]
            elif j ==3:
                days += month_arr[i][j]

    # -------------------- Tallying up the time ----------------------- # 
    # need to adapt this to the array format
    count_mins = 0

    bflag = True
    while bflag:
        if seconds - 60 > 60:
            count_mins +=1
            seconds -= 60
        elif seconds > 60:
            count_mins +=1
            seconds -= 60
            bflag = False
        else:
            break

    minutes += count_mins

    count_hours = 0
    bflag = True
    while bflag:
        if minutes - 60 > 60:
            count_hours +=1
            minutes -= 60
        elif minutes > 60:
            count_hours +=1
            minutes -= 60
            bflag = False
        else:
            break

    hours += count_hours

    count_days = 0
    bflag = True
    while bflag:
        if hours - 24 > 24:
            count_days +=1
            hours -= 24
        elif hours > 24:
            count_days +=1
            hours -= 24
            bflag = False
        else:
            break

    days += count_days
    # ----------------- End Tallying up the time -------------------- #
    return (days*24 + hours + minutes/60 + seconds/3600 )

In [68]:
no_activities = len(data['activities'])

sec_arr = []
min_arr = []
hour_arr = []
day_arr = []
combo_arr = []
activity_arr = []

#------ Month Arrays -------#
# Can be made more modular
jan_arr = []
feb_arr = []
mar_arr = []
apr_arr = []
may_arr = []
jun_arr = []
jul_arr = []
aug_arr = []
sep_arr = []
oct_arr = []
nov_arr = []
dec_arr = []


for j in range(no_activities):
    
    activity = data['activities'][j]['name']
    
    # Retrieving all data that specific things were worked on
    time_entry_data = (data['activities'][j]['time_entries'])
    
    for i in range(len(time_entry_data)):
        
        month = str(time_entry_data[i]['start_time'][5:7])
        if (month) == '01':
            jan_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '02':
            feb_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '03':
            mar_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '04':
            apr_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '05':
            may_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '06':
            jun_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '07':
            jul_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '08':
            aug_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '09':
            sep_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '10':
            oct_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '11':
            nov_arr.append(timeHandler(time_entry_data,i)) 
        elif (month) == '12':
            dec_arr.append(timeHandler(time_entry_data,i)) 

hours_per_month = [
    monthTimeHandler(jan_arr),
    monthTimeHandler(feb_arr),
    monthTimeHandler(mar_arr),
    monthTimeHandler(apr_arr),
    monthTimeHandler(may_arr),
    monthTimeHandler(jun_arr),
    monthTimeHandler(jul_arr),
    monthTimeHandler(aug_arr),
    monthTimeHandler(sep_arr),
    monthTimeHandler(oct_arr),
    monthTimeHandler(nov_arr),
    monthTimeHandler(dec_arr),
]    

charged_per_month = [
    monthTimeHandler(jan_arr)*rate,
    monthTimeHandler(feb_arr)*rate,
    monthTimeHandler(mar_arr)*rate,
    monthTimeHandler(apr_arr)*rate,
    monthTimeHandler(may_arr)*rate,
    monthTimeHandler(jun_arr)*rate,
    monthTimeHandler(jul_arr)*rate,
    monthTimeHandler(aug_arr)*rate,
    monthTimeHandler(sep_arr)*rate,
    monthTimeHandler(oct_arr)*rate,
    monthTimeHandler(nov_arr)*rate,
    monthTimeHandler(dec_arr)*rate,
]    

In [69]:
dataf = {
    'Months': ['Jan',
               'Feb',
               'Mar',
               'Apr',
               'May',
               'Jun',
               'Jul',
               'Aug',
               'Sep',
               'Oct',
               'Nov',
               'Dec'],
    'Hours Per Month': hours_per_month,
    'Charge': charged_per_month
}

df_total = pd.DataFrame(dataf)
df_total

,Months,Hours Per Month,Charge
0,Jan,0.014167,1.416667
1,Feb,0.222778,22.277778
2,Mar,0.000000,0.000000
3,Apr,0.000000,0.000000
4,May,0.000000,0.000000
5,Jun,0.000000,0.000000
6,Jul,0.000000,0.000000
7,Aug,0.000000,0.000000
8,Sep,0.000000,0.000000
9,Oct,0.000000,0.000000


In [70]:
fig = px.bar(df_total, x='Months', y='Hours Per Month',text='Charge')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, 
                  uniformtext_mode='hide',
                  title = 'Hours worked per month')
fig.show()

In [71]:
fig = px.pie(df_total, 
             values='Hours Per Month', 
             names='Months',
             title=f'Time Spent per month: {total_Hours_Worked}')
fig.show()